In [ ]:
!pip install pygame

import pygame
import random
import time

pygame.init()

width, height = 1200, 800
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("순번 사다리")

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

TRACE_COLORS = [
    (255, 128, 128), (128, 255, 128), (0, 0, 255), (120, 120, 128), (120, 128, 255),
    (0, 255, 255), (128, 0, 0), (0, 128, 0), (0, 0, 128), (128, 128, 0)
]

font = pygame.font.SysFont("에스코어드림8heavy", 36) # 시스템 폰트에서 선택

num_participants = int(input("참여자 수(2-10): "))
while num_participants < 2 or num_participants > 10:
    num_participants = int(input("참여자 수는 2에서 10 사이여야 합니다. 다시 입력하세요: "))

inputs = []
for i in range(num_participants):
    value = input(f"조 이름: ")
    inputs.append(value)

results = list(range(1, num_participants + 1))
random.shuffle(results)

column_spacing = (width - 200) // (num_participants - 1)

ladder = []
horizontal_lines = []
for i in range(num_participants):
    ladder.append([(100 + i * column_spacing, 100), (100 + i * column_spacing, 500)])

for i in range(10):
    y = 150 + i * 35
    start = random.randint(0, num_participants - 2)
    horizontal_lines.append((start, y))
    ladder.append([(100 + start * column_spacing, y), (100 + (start + 1) * column_spacing, y)])

def draw_ladder():
    for line in ladder:
        pygame.draw.line(screen, BLACK, line[0], line[1], 2)

def find_path(start_col):
    x = 100 + start_col * column_spacing
    y = 100
    path = [(x, y)]
    current_col = start_col

    while y < 500:
        y += 1
        for col, h_y in horizontal_lines:
            if y == h_y:
                if col == current_col:
                    current_col += 1
                    while x < 100 + current_col * column_spacing:
                        x += 1
                        path.append((x, y))
                elif col == current_col - 1:
                    current_col -= 1
                    while x > 100 + current_col * column_spacing:
                        x -= 1
                        path.append((x, y))
        path.append((x, y))

    return path, current_col

running = True
paths = [find_path(i) for i in range(num_participants)]
current_path = 0
path_index = 0
final_results = []
traces = [[] for _ in range(num_participants)]

clock = pygame.time.Clock()

show_final_results = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    screen.fill(WHITE)
    
    if not show_final_results:
        draw_ladder()
        
        # 자취 그리기
        for i, trace in enumerate(traces):
            if len(trace) >= 2:
                pygame.draw.lines(screen, TRACE_COLORS[i], False, trace, 4)
            elif len(trace) == 1:
                pygame.draw.circle(screen, TRACE_COLORS[i], trace[0], 2)
        
        for i, value in enumerate(inputs):
            text = font.render(value, True, BLACK)
            text_rect = text.get_rect(center=(100 + i * column_spacing, 50))
            screen.blit(text, (70 + i * column_spacing, 50))
        
        for i, result in enumerate(results):
            text = font.render(str(result), True, RED)
            text_rect = text.get_rect(center=(100 + i * column_spacing, 550))
            screen.blit(text, (90 + i * column_spacing, 550))
        
        if current_path < num_participants:
            if path_index < len(paths[current_path][0]):
                x, y = paths[current_path][0][path_index]
                traces[current_path].append((x, y))
                pygame.draw.circle(screen, TRACE_COLORS[current_path], (x, y), 5)
                path_index += 1
            else:
                final_results.append((results[paths[current_path][1]], inputs[current_path]))
                path_index = 0
                current_path += 1
                time.sleep(0.5)
        else:
            show_final_results = True
    else:
        title = font.render("Result", True, BLACK)
        title_rect = title.get_rect(center=(width // 2, 50))
        screen.blit(title, (width // 2 - 50, 50))
        
        final_results.sort()
        for i, (num, name) in enumerate(final_results):
            text = font.render(f"No.{num} : {name}", True, TRACE_COLORS[i])
            text_rect = text.get_rect(center=(width//2, 150 + i * 50))
            screen.blit(text, (width // 2 - 100, 150 + i * 50))

    pygame.display.flip()
    clock.tick(600)

pygame.quit()